In [1]:
import pygame
import time
import random
from scipy.spatial import distance 
import cProfile

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Entorno:
    def __init__(self, tipo, posicion, energia,recuperacion):
        self.tipo = tipo
        self.posicion = posicion  # (x, y) tuple
        self.energia = energia
        self.recuperacion = int(recuperacion/4)
        self.recuperacion_maxima = recuperacion
        
    def disponible(self):
        return self.recuperacion == 0

    def cooldown(self):
        if self.recuperacion > 0:
            self.recuperacion -= 1
    def __str__(self):
        return f"_________ Tipo: {self.tipo}, Posición: {self.posicion}, CD: {self.recuperacion}, Energía: {self.energia}"

In [3]:
class Organismo:
    def __init__(self, tipo, posicion, energia):
        self.tipo = tipo
        self.posicion = posicion  # (x, y)
        self.energia = energia
        self.energia_maxima = 150  # Energía máxima que puede tener el organismo

    def mover(self):
        # Movimiento aleatorio en una cuadrícula
        if self.tipo == "conejo":
            best,objetivo = 9999,0
            for entorno in mundo.entornos:
                if entorno.tipo == "arbusto" and self.radio_busqueda(entorno,25):
                    distancia = distance.euclidean(self.posicion, entorno.posicion)
                    cooldown = entorno.recuperacion
                    energia = entorno.energia
                    puntos = distancia+(cooldown*3.5)-(energia*.6)
                    if puntos < best:
                        best = puntos
                        objetivo = entorno.posicion
            if objetivo == 0:
                dx = random.choice([-5, 0, 5])
                dy = random.choice([-5, 0, 5])
            else:
                res = [objetivo[i]-self.posicion[i] for i in range(2)]
                direccion = [-5 if i < 0 else 5 if i > 0 else 0 for i in res]
                dx,dy = direccion[0],direccion[1]

        if self.tipo == "zorro":
            best,objetivo = 9999,0
            for organismo in mundo.organismos:
                if organismo.tipo == "conejo" and self.radio_busqueda(organismo, 35):
                    distancia = distance.euclidean(self.posicion,organismo.posicion)
                    energia = organismo.energia
                    puntos = distancia+energia*.4
                    if puntos < best:
                        best = puntos
                        objetivo = organismo.posicion
            if objetivo == 0:
                dx = random.choice([-10, -5, 0, 5, 10])
                dy = random.choice([-10, -5, 0, 5, 10])
            else:
                res = [objetivo[i]-self.posicion[i] for i in range(2)] #aqui falta poner el -10 y el 10 
                direccion = [-5 if i < 0 else 5 if i > 0 else 0 for i in res]
                dx,dy = direccion[0],direccion[1]
            
        self.posicion = (self.posicion[0] + dx, self.posicion[1] + dy)
        if dx == 0 and dy == 0:
            self.energia = round(self.energia - .2,1)
        else:
            self.energia = round(self.energia - 1,1)
            
    def alimentarse(self, mundo):
        # Los conejos Se alimentan de arbustos adyacentes
        if self.tipo == "conejo":
            for entorno in mundo.entornos:
                if entorno.tipo == "arbusto" and self.radio_busqueda(entorno,5) and entorno.disponible():
                    self.energia = min(self.energia + entorno.energia + random.randint(-5, 5), self.energia_maxima)
                    entorno.recuperacion = entorno.recuperacion_maxima
                    break

    def reproducirse(self):
        # Se reproduce si la energía es mayor que un umbral
        if self.energia > 80:
            self.energia //= 2  # Reduce la energía a la mitad
            hijo_posicion = (self.posicion[0] + random.choice([-5, 5]),
                             self.posicion[1] + random.choice([-5, 5]))
            return Organismo(self.tipo, hijo_posicion, self.energia)
        return None

    def cazar(self, mundo):
        # Los zorros cazan conejos adyacentes
        if self.tipo == "zorro":
            for organismo in mundo.organismos:
                if organismo.tipo == "conejo" and self.radio_busqueda(organismo,10):
                    self.energia = min(self.energia + organismo.energia + random.randint(0, 25), self.energia_maxima)
                    mundo.organismos.remove(organismo)
                    break
    def morir(self):
        if self.energia <= 0:
            mundo.organismos.remove(self)

    
    def radio_busqueda(self, otro_organismo,radio):
        dx = abs(self.posicion[0] - otro_organismo.posicion[0])
        dy = abs(self.posicion[1] - otro_organismo.posicion[1])
        return dx <= radio and dy <= radio

    def __str__(self):
        return f"Tipo: {self.tipo}, Posición: {self.posicion}, Energía: {self.energia}"


In [4]:
class Mundo:
    def __init__(self, tamaño):
        self.tamaño = tamaño  # (ancho, alto)
        self.organismos = []
        self.entornos = []
        self.ciclos = 0

    def agregar_organismo(self, organismo):
        self.organismos.append(organismo)

    def agregar_entorno(self, entorno):
        self.entornos.append(entorno)
    
    def actualizar(self):
        self.ciclos+=1
        nuevos_organismos = []
        for entorno in self.entornos:
            entorno.cooldown()
        for organismo in self.organismos:
            organismo.mover()
            organismo.alimentarse(self)
            organismo.cazar(self)
            organismo.morir()
            nuevo_organismo = organismo.reproducirse()
            if nuevo_organismo:
                nuevos_organismos.append(nuevo_organismo)
        self.organismos.extend(nuevos_organismos)

    def mostrar_estado(self):
        print(f"--- ciclo {self.ciclos} ---")
        for organismo in self.organismos:
            print(organismo)
        for entorno in self.entornos:
            print(entorno)
# # Ejecutar la simulación
# for paso in range(30):
#     print(f"Paso {paso + 1}:")
#     mundo.actualizar()
#     mundo.mostrar_estado()
#     print("---")


In [5]:
# Inicializar el mundo y agregar organismos
mundo = Mundo(tamaño=(20, 20))
organismos = []
organismos.append(Organismo(tipo="conejo", posicion=(100, 100), energia=45))
organismos.append(Organismo(tipo="conejo", posicion=(70, 170), energia=45))
organismos.append(Organismo(tipo="zorro", posicion=(120, 130), energia=70))

entornos = []
for i in range(15):
    n = random.randint(0,2)
    entornos.append(Entorno(tipo="arbusto",posicion=(random.randint(10, 40)*5,random.randint(10, 40)*5),energia=[20,70,120][n],recuperacion=[20,55,80][n]))  

for org in organismos:
    mundo.agregar_organismo(org)
for ent in entornos:
    mundo.agregar_entorno(ent)

In [106]:
mundo.mostrar_estado()
mundo.actualizar()
mundo.mostrar_estado()

--- ciclo 12 ---
Tipo: conejo, Posición: (110, 105), Energía: 41.0
Tipo: conejo, Posición: (65, 190), Energía: 36.2
Tipo: zorro, Posición: (120, 165), Energía: 58.8
_________ Tipo: arbusto, Posición: (190, 65), CD: 8, Energía: 120
_________ Tipo: arbusto, Posición: (165, 165), CD: 0, Energía: 20
_________ Tipo: arbusto, Posición: (175, 195), CD: 0, Energía: 20
_________ Tipo: arbusto, Posición: (75, 105), CD: 0, Energía: 20
_________ Tipo: arbusto, Posición: (140, 185), CD: 8, Energía: 120
_________ Tipo: arbusto, Posición: (110, 105), CD: 1, Energía: 70
_________ Tipo: arbusto, Posición: (100, 175), CD: 0, Energía: 20
_________ Tipo: arbusto, Posición: (105, 190), CD: 0, Energía: 20
_________ Tipo: arbusto, Posición: (195, 125), CD: 8, Energía: 120
_________ Tipo: arbusto, Posición: (135, 90), CD: 1, Energía: 70
_________ Tipo: arbusto, Posición: (90, 50), CD: 8, Energía: 120
_________ Tipo: arbusto, Posición: (195, 135), CD: 1, Energía: 70
_________ Tipo: arbusto, Posición: (165, 95)

In [36]:
pygame.init()
pantalla = pygame.display.set_mode((200, 200))
clock = pygame.time.Clock()
running = True

while running:
    clock.tick(60)
    for evento in pygame.event.get():
        if evento.type == pygame.QUIT:
            running = False
            pygame.quit()
    if not running:
        break

    pantalla.fill((255, 255, 255))
    for entorno in mundo.entornos:
        pygame.draw.circle(pantalla, (128, 255, 128), entorno.posicion, 2.5)
    for organismo in mundo.organismos:
        if organismo.tipo == "conejo":
            pygame.draw.circle(pantalla, (0, 0, 255), organismo.posicion, 2.5)
        else:
            pygame.draw.circle(pantalla, (255, 0, 0), organismo.posicion, 2.5)
    pygame.display.flip()
    mundo.mostrar_estado()
    mundo.actualizar()
    time.sleep(1)
print("Salida")

--- ciclo 1 ---
Tipo: conejo, Posición: (95, 100), Energía: 44
Tipo: zorro, Posición: (110, 125), Energía: 69
_________ Tipo: arbusto, Posición: (90, 100), Recuperación: 4 
_________ Tipo: arbusto, Posición: (90, 110), Recuperación: 4 
_________ Tipo: arbusto, Posición: (90, 90), Recuperación: 4 
_________ Tipo: arbusto, Posición: (100, 110), Recuperación: 4 
_________ Tipo: arbusto, Posición: (100, 90), Recuperación: 4 
_________ Tipo: arbusto, Posición: (110, 100), Recuperación: 4 
_________ Tipo: arbusto, Posición: (110, 110), Recuperación: 4 
_________ Tipo: arbusto, Posición: (110, 90), Recuperación: 4 
_________ Tipo: arbusto, Posición: (120, 115), Recuperación: 11 
--- ciclo 2 ---
Tipo: conejo, Posición: (90, 100), Energía: 43
Tipo: zorro, Posición: (100, 130), Energía: 68
_________ Tipo: arbusto, Posición: (90, 100), Recuperación: 3 
_________ Tipo: arbusto, Posición: (90, 110), Recuperación: 3 
_________ Tipo: arbusto, Posición: (90, 90), Recuperación: 3 
_________ Tipo: arbus